In [1]:
#poblar la variable de dataset y una variable de subset de 1000 registros para correr pruebas
import pandas as pd
import numpy as np

df = pd.read_csv('crimestats.csv', header=0)
subset1000 = df[:1001]


In [2]:
#Analizar el dataset y sus estadisticos descriptivos
stats = df.describe(include='all')
print(stats)

                      Dates       Category                      Descript  \
count                878049         878049                        878049   
unique               389257             39                           879   
top     2011-01-01 00:01:00  LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   
freq                    185         174900                         60022   
mean                    NaN            NaN                           NaN   
std                     NaN            NaN                           NaN   
min                     NaN            NaN                           NaN   
25%                     NaN            NaN                           NaN   
50%                     NaN            NaN                           NaN   
75%                     NaN            NaN                           NaN   
max                     NaN            NaN                           NaN   

       DayOfWeek PdDistrict Resolution                 Address              X  \
count 

In [ ]:
#dibujar pareto de categorias para identificar las mas significativas
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="ticks")

cat_count = pd.DataFrame(df.Category.value_counts())
cat_count['cumpercentage'] = cat_count['Category'].cumsum()/cat_count['Category'].sum()*100

fig, ax1 = plt.subplots(figsize=(10,6))
ax1.set_title('Cantidad de crimenes por categoria, 2003-2015', fontsize=16)
ax1.set_xlabel('Categoria', fontsize=12)
ax2 = sns.barplot(x=cat_count.index, y=cat_count.Category, data = cat_count, palette='flare')
ax1.set_ylabel('Cantidad de crimenes' , fontsize = 12)
ax2 = ax1.twinx()
color = 'tab:blue'
ax2 = sns.lineplot(x=cat_count.index, y=cat_count.cumpercentage, data = cat_count, sort=False, color=color)
ax2.set_ylabel('Frecuencia(%) acumulada de crimenes', fontsize = 12)
ax2.axhline(80, ls='--')
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
#extraer las categorias que agrupan el 80% de los crimenes para una visualizacion mas limpia
top80 = cat_count.index[:12]
newdf = df[df.Category.isin(top80)]



In [4]:

df['Dates'] = pd.to_datetime(df['Dates'])
df['Year'] = pd.DatetimeIndex(df['Dates']).year
df['Month'] = pd.DatetimeIndex(df['Dates']).month
df['Hour'] = pd.DatetimeIndex(df['Dates']).hour + pd.DatetimeIndex(df['Dates']).minute / 60
df['timestamp'] = (df['Dates'] - df['Dates'].min())  / np.timedelta64(1,'D')

df_cat = df.select_dtypes(object)
df_num = df.select_dtypes(np.number)

df_cat.drop(['Descript','Address'], axis=1, inplace=True)
df_cat.head()



,Category,DayOfWeek,PdDistrict,Resolution
0,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED"
1,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED"
2,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED"
3,LARCENY/THEFT,Wednesday,NORTHERN,NONE
4,LARCENY/THEFT,Wednesday,PARK,NONE


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_cat = df_cat.apply(le.fit_transform)


In [6]:
df_final = pd.concat([df_cat,df_num],axis=1)

In [7]:
X = df_final.drop(['Category'], axis=1)
Y = df_final['Category']
X_train = np.array(X[0:int(0.80*len(X))])
Y_train = np.array(Y[0:int(0.80*len(Y))])
X_test = np.array(X[int(0.80*len(X)):])
Y_test = np.array(Y[int(0.80*len(Y)):])

In [ ]:
from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.svm import LinearSVC
#from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

LR = LogisticRegression()
#KNN = KNeighborsClassifier()
#NB = GaussianNB()
#LSVM = LinearSVC()
#NLSVM = SVC(kernel='rbf')
#DT = DecisionTreeClassifier()
#RF = RandomForestClassifier()

LR_fit = LR.fit(X_train, Y_train)
#KNN_fit = KNN.fit(X_train, Y_train)
#NB_fit = NB.fit(X_train, Y_train)
#LSVM_fit = LSVM.fit(X_train, Y_train)
#NLSVM_fit = NLSVM.fit(X_train, Y_train)
#DT_fit = DT.fit(X_train, Y_train)
#RF_fit = RF.fit(X_train, Y_train)

LR_pred = LR_fit.predict(X_test)
#KNN_pred = KNN_fit.predict(X_test)
#NB_pred = NB_fit.predict(X_test)
#LSVM_pred = LSVM_fit.predict(X_test)
#NLSVM_pred = NLSVM_fit.predict(X_test)
#DT_pred = DT_fit.predict(X_test)
#RF_pred = RF_fit.predict(X_test)

print("Logistic Regression is %f percent accurate" % (accuracy_score(LR_pred, Y_test)*100))
#print("KNN is %f percent accurate" % (accuracy_score(KNN_pred, Y_test)*100))
#print("Naive Bayes is %f percent accurate" % (accuracy_score(NB_pred, Y_test)*100))
#print("Linear SVMs is %f percent accurate" % (accuracy_score(LSVM_pred, Y_test)*100))
#print("Non Linear SVMs is %f percent accurate" % (accuracy_score(NLSVM_pred, Y_test)*100))
#print("Decision Trees is %f percent accurate" % (accuracy_score(DT_pred, Y_test)*100))
#print("Random Forests is %f percent accurate" % (accuracy_score(RF_pred, Y_test)*100))

In [ ]:
import folium
import webbrowser

latitude = 37.773972
longitude = -122.431297
crime_map = folium.Map(location=[latitude, longitude], zoom_start=5)